In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, avg, count, desc, floor, corr
import numpy as np
import time

spark_session = SparkSession.builder\
        .master("spark://192.168.2.223:7077") \
        .appName("Song Analysis with Decade Tempo")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
sqlContext = SQLContext(spark_session.sparkContext) 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 14:44:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
start_time = time.time()

In [3]:
# Load the dataset
df = sqlContext.read.csv('hdfs://localhost:9000/user/ubuntu/MillionSongSubset_aggregated_2GB.csv',
                                       header='true', inferSchema='true').cache()

df.show()

+----+-------+---------+
|Year|  Tempo| Duration|
+----+-------+---------+
|2008|124.059|148.74077|
|2004| 80.084|252.99546|
|2007|  77.15|163.63057|
|1982| 92.897|  216.842|
|1998| 86.981|312.99873|
|2004|157.715|218.90567|
|2005|146.331|580.70159|
|2004| 84.992|283.48036|
|2001| 99.959| 135.8624|
|1982|104.989|146.33751|
|2000|105.095|208.95302|
|1981|100.042|154.93179|
|2004| 80.893|146.80771|
|1973|  110.1|199.96689|
|2007| 92.971|319.84281|
|2008|117.936|258.16771|
|1963|121.896|207.35955|
|2001| 152.99|258.29832|
|2004|104.465| 158.1971|
|2004|138.512|461.71383|
+----+-------+---------+
only showing top 20 rows



In [4]:
# Filter out rows with non-values or values <= 0 for Year, Tempo, and Duration
df_filtered = df.filter((col("Year") > 1970) & (col("Tempo") > 0) & (col("Duration") > 0))

In [5]:
# Calculate decade and average Tempo by Decade
df_with_decade = df_filtered.withColumn("Decade", (floor(df_filtered["Year"] / 10) * 10))
df_decade_tempo = df_with_decade.groupBy("Decade").agg(
    avg("Tempo").alias("average_tempo"), 
    avg("Duration").alias("average_duration"), 
    count("*").alias("songs_count")).orderBy("Decade")

In [6]:
df_nondecade_tempo = df_with_decade.groupBy("Year").agg(
    avg("Tempo").alias("average_tempo"), 
    avg("Duration").alias("average_duration"), 
    count("*").alias("songs_count")).orderBy("Year")

df_nondecade_tempo_filtered = df_nondecade_tempo.filter(df_nondecade_tempo["songs_count"] > 5).orderBy("Year")

In [7]:
df_decade_tempo.show()

+------+------------------+------------------+-----------+
|Decade|     average_tempo|  average_duration|songs_count|
+------+------------------+------------------+-----------+
|  1970|131.70000444444665| 250.6536162666558|    4610025|
|  1980|126.86822425620393|235.72452128144687|    8953693|
|  1990|124.63248459611607|242.58668049975944|   24607289|
|  2000|125.99748443056102| 239.6860924792395|   51980593|
|  2010|118.92348437499787| 238.0424034375433|    1311296|
+------+------------------+------------------+-----------+



In [8]:
df_nondecade_tempo_filtered.show()

+----+------------------+------------------+-----------+
|Year|     average_tempo|  average_duration|songs_count|
+----+------------------+------------------+-----------+
|1971|136.16195999999766|259.55428920000645|     512225|
|1972|129.17204166668014|238.54539749999654|     491736|
|1973|116.35612500000175| 294.1644441666234|     491736|
|1974|125.08609090909557| 239.4913463636304|     450758|
|1975|125.41183333332964|  277.440635416734|     491736|
|1976|137.26140000000402|210.99404933333955|     307335|
|1977|139.33685714286017| 255.3069279999802|     717115|
|1978| 134.3838499999925| 247.8545675000161|     409780|
|1979|137.51694444442083|226.05668861112719|     737604|
|1980|126.89337500000521|210.43873062496715|     655648|
|1981| 131.6167714285617|217.70552914285332|     717115|
|1982|125.14521999998475| 251.2374242000137|    1024450|
|1983|126.14368085104255|237.46428063834134|     962983|
|1984|127.95624999999713|239.31872999999382|     655648|
|1985|132.86585294118507|242.61

In [9]:
correlation_df = df_filtered.select(
    corr("Tempo", "Year").alias("tempo_year_corr"),
    corr("Duration", "Year").alias("duration_year_corr")
)

correlation_df.show()

+--------------------+--------------------+
|     tempo_year_corr|  duration_year_corr|
+--------------------+--------------------+
|-0.02507322891847...|-0.01188938532914...|
+--------------------+--------------------+



In [10]:
correlation_decade_df = df_with_decade.select(
    corr("Tempo", "Decade").alias("tempo_decade_corr"),
    corr("Duration", "Decade").alias("duration_decade_corr")
)

correlation_decade_df.show()

+--------------------+--------------------+
|   tempo_decade_corr|duration_decade_corr|
+--------------------+--------------------+
|-0.02702604138343866|-0.01303098492626...|
+--------------------+--------------------+



In [11]:
print("Time taken: ", time.time() - start_time)

Time taken:  75.46774959564209


In [12]:
spark_session.stop()